In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

In [2]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Fetches an image online, resizes it and saves it locally.
    
    Args:
        url (string) -- link to the image
        new_width (int) -- size in pixels used for resizing the width of the image
        new_height (int) -- size in pixels used for resizing the length of the image
        
    Returns:
        (string) -- path to the saved image
    '''
    
    
    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # opens the given URL
    response = urlopen(url)
    
    # reads the image fetched from the URL
    image_data = response.read()
    
    # puts the image data in memory buffer
    image_data = BytesIO(image_data)
    
    # opens the image
    pil_image = Image.open(image_data)
    
    # resizes the image. will crop if aspect ratio is different.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    
    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")
    
    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Image downloaded to %s." % filename)
    
    return filename

In [3]:
# you can switch the commented lines here to pick the other model

# inception resnet version 2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# You can choose ssd mobilenet version 2 instead and compare the results
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

In [4]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction pruned(images) at 0x7FBE401B6650>}))

In [6]:
detector = model.signatures['default']

In [7]:
# You can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmppjm2wj0_.jpg.


In [8]:
def load_img(path):
    '''
    Loads a JPEG image and converts it to a tensor.
    
    Args:
        path (string) -- path to a locally saved JPEG image
    
    Returns:
        (tensor) -- an image tensor
    '''
    
    # read the file
    img = tf.io.read_file(path)
    
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img

In [11]:
def run_detector(detector,path):
  img = load_img(path)

  converted_img = tf.image.convert_image_dtype(img,tf.float32)[tf.newaxis,...]
  result = detector(converted_img)

  result = {key:value.numpy() for key,value in result.items()}

  # print results
  print("Found %d objects." % len(result["detection_scores"]))

  print(result["detection_scores"])
  print(result["detection_class_entities"])
  print(result["detection_boxes"])

In [12]:
run_detector(detector,downloaded_image_path)

Found 100 objects.
[0.65321743 0.6105046  0.6015251  0.59255654 0.5917774  0.5815494
 0.5505317  0.495756   0.47424808 0.47322178 0.44066453 0.405114
 0.39803833 0.39406568 0.37148643 0.36156058 0.36150774 0.34689203
 0.3336231  0.3125288  0.28877988 0.257584   0.25748932 0.25196046
 0.2478206  0.23412406 0.2043204  0.20324677 0.17988251 0.17964818
 0.1737451  0.1643149  0.16031115 0.15895244 0.15620375 0.15468794
 0.14754476 0.1362262  0.12740143 0.12555654 0.12102691 0.11813018
 0.11387325 0.11229103 0.11129177 0.09718708 0.09137247 0.08976
 0.08880243 0.08633528 0.08337455 0.0809516  0.07988811 0.07741416
 0.07732146 0.07631036 0.07507854 0.07386072 0.07233327 0.07204052
 0.07110188 0.06935578 0.06825594 0.06427866 0.06248543 0.06226372
 0.0621115  0.05940033 0.05798788 0.05784449 0.05725611 0.05346696
 0.05304365 0.05242864 0.04892578 0.04813037 0.04578469 0.04424176
 0.04337554 0.0427781  0.04262635 0.04163501 0.0408176  0.03976011
 0.03945978 0.03944282 0.03863983 0.03769045 0.03